In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# If the icetemperature library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '../'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)

# Import the ice temperature model and relevant constants
from icetemperature.lib.numerical_model import ice_temperature
from icetemperature.lib.analytical_solutions import Robin_T, Rezvan_T, Meyer_T, Perol_T
from icetemperature.lib.ice_properties import conductivity, heat_capacity, rate_factor
from icetemperature.lib.constants import constants
const = constants()

In [ ]:
m = ice_temperature(nz=201)
#m.flags.pop()
m.initial_conditions()
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()

plt.figure()
ax1 = plt.subplot(121)
ax1.set_ylabel('Height (m)')
ax1.set_xlabel('Temp ($^\circ$C)')
plt.text(.1,.1,'A',transform=ax1.transAxes,fontweight='bold',fontsize=12)
ax2 = plt.subplot(122)
ax2.tick_params(labelleft=False)
ax2.set_xlabel('Vertical Velocity (m/yr)')
plt.text(.8,.1,'B',transform=ax2.transAxes,fontweight='bold',fontsize=12)
plt.tight_layout()

ax1.plot(Robin_T(m),m.z,'k-',lw=2,label='Robin (1955)')
ax1.plot(m.T,m.z,'r--',label='Numerical p=1000')
ax2.plot(m.v_z*const.spy,m.z,'r--')

m.p = 5.
m.initial_conditions()
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()

ax1.plot(m.T,m.z,'r-',label='p=%.0f'%m.p)
ax2.plot(m.v_z*const.spy,m.z,'r-')
ax1.legend()

plt.savefig('Robin_reproduce_analytical.png',dpi=300)

## Rezban Figure 2

In [ ]:
from matplotlib import cm
%matplotlib inline

plt.figure(figsize=(10,4))
ax1 = plt.subplot(131)
plt.xlabel('Vertical Velocity (m/yr)')
plt.ylabel('Height (m)')
ax2 = plt.subplot(132)
plt.xlabel('Temp ($^\circ$C)')
ax3 = plt.subplot(133)
plt.xlabel('Analytical - Numerical ($^\circ$C)')

m = ice_temperature(adot=0.3,H=3000,p=None,tol=1e-6)
m.flags.pop()

gammas = np.linspace(1.41,1.77,13)
for i,gamma in enumerate(gammas):
    m.gamma = gamma
    m.initial_conditions(analytical=Robin_T)
    m.source_terms()
    m.stencil(dt=1.*const.spy)
    m.numerical_to_steady_state()

    RezT = Rezvan_T(m,T_bulk=None)
    ax2.plot(RezT,m.z,c=cm.viridis(i/len(gammas)))
    ax1.plot(-m.v_z*const.spy,m.z,c=cm.viridis(i/len(gammas)),label='γ=%1.2f'%m.gamma)
    ax3.plot(RezT-m.T,m.z,c=cm.viridis(i/len(gammas)))

ax1.legend(ncol=2,loc=3,fontsize=8)
plt.tight_layout()

plt.savefig('Rezvan_reproduce_analytical.png',dpi=300)

## Meyer Figure 1

In [ ]:
H = 1000.
Tb = 0.
lam = 0.
Ts = -25.
dT = Tb - Ts
A = 2.4e-24

plt.figure(figsize=(6,8))
ax1 = plt.subplot(211)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylabel('Height (m)')
ax2 = plt.subplot(212)
plt.xlabel('Br-Λ')
plt.ylabel('Pe')

Pe = 2.
Br = 4.
eps_xy = (Br*(const.k*dT)/(2.*H**2.)*A**(1./const.n))**(const.n/(const.n+1.))
adot = const.k/(const.rho*const.Cp)*Pe/H
m = ice_temperature(Ts=-25.,H=1000.,adot=adot*const.spy,eps_xy=eps_xy*const.spy)
m.flags.pop()
ax1.plot(Meyer_T(m,T_bulk=0.),m.z,'r',label='Pe=2; Br=4')

Pe = 0.
Br = 4.
eps_xy = (Br*(const.k*dT)/(2.*H**2.)*A**(1./const.n))**(const.n/(const.n+1.))
adot = const.k/(const.rho*const.Cp)*Pe/H
m = ice_temperature(Ts=-25.,H=1000.,adot=adot*const.spy,eps_xy=eps_xy*const.spy)
m.flags.pop()
ax1.plot(Meyer_T(m,T_bulk=0.),m.z,'b',label='Pe=0; Br=4')


nP,nB = 200,200
tol = 1e-2
h_bars = np.empty((nP,nB))
Pes = np.linspace(0,10,nP)
Brs = np.linspace(0,10,nB)
for ip,Pe in enumerate(Pes):
    for ib,Br in enumerate(Brs):
        eps_xy = (Br*(const.k*dT)/(2.*H**2.)*A**(1./const.n))**(const.n/(const.n+1.))
        adot = const.k/(const.rho*const.Cp)*Pe/H
        m = ice_temperature(Ts=-25.,H=1000.,adot=adot*const.spy,eps_xy=eps_xy*const.spy)
        MeyT = Meyer_T(m,T_bulk=0.)
        if not np.any(abs(MeyT)<tol):
            h_bars[ip,ib] = 0.
        else:
            h_bars[ip,ib] = np.max(m.z[abs(MeyT)<tol])/m.H
            
im = ax2.contourf(Brs,Pes,h_bars,levels=100)
plt.colorbar(im,ax=ax2,label='ξ/H')
ax2.contour(Brs,Pes,h_bars,levels=[0.0],colors='k',linewidths=2.)

plt.savefig('Meyer_reproduce_analytical.png',dpi=300)

## Perol Figure 3

In [ ]:
from matplotlib import cm
%matplotlib inline

# Constants from Perol
Margins = ['A','WB1','WB2','W Narrows','W Plain','TWB1','TWB2','C','TC1','TC2','D','TD1','TD2','TD3','E','TE']
str_heat_arr = np.array([4.2,7,9.5,13.5,5.1,3.8,4.0,1.,1.4,0.9,5.8,2.5,5.4,2.2,8.1,5.5])*1e-2/2.
H_arr = np.array([1242.,1205.,985.,846.,735.,2188.,1538.,1802.,1802.,2196.,888.,1952.,1412.,1126.,916.,1177.])

plt.figure(figsize=(8,4))
ax1 = plt.subplot(121)
ax1.set_xlim(-30,20)
ax1.set_ylim(0,1400)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylabel('Analytical - Numerical ($^\circ$C)')
ax2 = plt.subplot(122)
plt.xlabel('Analytical - Numerical ($^\circ$C)')

idxs = [0,1,2,3,10,14]

for i,idx in enumerate(idxs):
    
    m = ice_temperature(Ts=-26.,H=H_arr[idx],adot=.1,eps_xy=str_heat_arr[idx])
    m.flags.pop()
    PerT = Perol_T(m,T_bulk='average')
    ax1.plot(PerT,m.z,'-',c=cm.viridis((i+1)/len(idxs)),label=Margins[idx])
    
    m.pmp = 100.*np.ones(m.nz)
    m.pmp[0] = const.rho*const.g*m.H*const.beta
    m.A_xy_init = rate_factor(-13.)
    m.flags.append('plane_strain')
    m.initial_conditions(analytical=Robin_T)
    m.k = conductivity(np.array([-13.]*m.nz),const.rho)
    m.Cp = heat_capacity(np.array([-13.]*m.nz))
    m.source_terms()
    m.stencil(dt=1.*const.spy)
    m.tol = 1e-6
    m.numerical_to_steady_state()
    ax2.plot(PerT-m.T,m.z,c=cm.viridis((i+1)/len(idxs)))
    
ax1.legend(ncol=2,loc=1)

plt.savefig('Perol_reproduce_analytical.png',dpi=300)

## Meyer (2018) model implemented at Siple Coast

In [ ]:
from matplotlib import cm
%matplotlib inline

# Constants from Perol
Margins = ['A','WB1','WB2','W Narrows','W Plain','TWB1','TWB2','C','TC1','TC2','D','TD1','TD2','TD3','E','TE']
str_heat_arr = np.array([4.2,7,9.5,13.5,5.1,3.8,4.0,1.,1.4,0.9,5.8,2.5,5.4,2.2,8.1,5.5])*1e-2/2.
H_arr = np.array([1242.,1205.,985.,846.,735.,2188.,1538.,1802.,1802.,2196.,888.,1952.,1412.,1126.,916.,1177.])

plt.figure(figsize=(8,4))
ax1 = plt.subplot(121)
ax1.set_xlim(-26,10)
ax1.set_ylim(0,1400)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylabel('Analytical - Numerical ($^\circ$C)')
ax2 = plt.subplot(122)
plt.xlabel('Analytical - Numerical ($^\circ$C)')
plt.xlim(-0.1,0.1)

idxs = [1,2,3]

for i,idx in enumerate(idxs):
    
    m = ice_temperature(Ts=-26.,H=H_arr[idx],adot=.1,eps_xy=str_heat_arr[idx])
    m.flags.pop()
    MeyT = Meyer_T(m,T_bulk=0.)
    ax1.plot(MeyT,m.z,'-',c=cm.viridis((i+1)/len(idxs)),label=Margins[idx])
    
    m.pmp = np.zeros(m.nz)
    m.A_xy_init = 'temperate'
    m.flags.append('plane_strain')
    m.initial_conditions(analytical=Robin_T)
    m.v_z[:] = m.v_z[-1]
    m.source_terms()
    m.stencil(dt=1.*const.spy)
    m.numerical_to_steady_state()
    ax2.plot(MeyT-m.T,m.z,c=cm.viridis((i+1)/len(idxs)))
    
ax1.legend(ncol=2,loc=1)

plt.savefig('Meyer_reproduce_Perol.png',dpi=300)

In [ ]:
plt.figure(figsize=(10,8))
ax0 = plt.subplot(341)
plt.ylabel('Height (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.text(.1,.1,'A',transform=ax0.transAxes,fontweight='bold',fontsize=12)
ax1 = plt.subplot(345)
plt.ylabel('Height (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.text(.1,.1,'B',transform=ax1.transAxes,fontweight='bold',fontsize=12)
ax2 = plt.subplot(346)
plt.xlabel('Temperature ($^\circ$C)')
plt.text(.1,.1,'C',transform=ax2.transAxes,fontweight='bold',fontsize=12)
ax3 = plt.subplot(347)
plt.xlabel('Temperature ($^\circ$C)')
plt.text(.1,.1,'D',transform=ax3.transAxes,fontweight='bold',fontsize=12)
ax4 = plt.subplot(348)
plt.xlabel('Temperature ($^\circ$C)')
plt.text(.1,.1,'E',transform=ax4.transAxes,fontweight='bold',fontsize=12)
axn = plt.subplot(349)
plt.ylabel('Height (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.text(.1,.1,'F',transform=axn.transAxes,fontweight='bold',fontsize=12)

Ts = -26.
eps_xy = 9.5*1e-2/2.
H = 1050.
adot = 0.0698
qgeo=0.0633

m = ice_temperature(Ts=Ts,H=H,adot=adot,eps_xy=eps_xy)
m.flags.pop()
MeyT = Meyer_T(m,T_bulk=0.)
ax0.plot(MeyT,m.z,'-',c='k',lw=2)
m.pmp = np.zeros(m.nz)
m.A_xy_init = 'temperate'
m.flags.append('plane_strain')
m.initial_conditions(analytical=Robin_T)
m.v_z[:] = m.v_z[-1]
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()
ax0.plot(m.T,m.z,c='r')
ax0.set_title('Reproduce Analytical',fontweight='bold')

m = ice_temperature(Ts=Ts,H=H,adot=adot,eps_xy=eps_xy)
m.flags.pop()
MeyT = Meyer_T(m,T_bulk=0.)
ax1.plot(MeyT,m.z,'-',c='k',lw=2)
m.pmp = np.zeros(m.nz)
m.A_xy_init = 'temperate'
m.flags.append('plane_strain')
m.initial_conditions(analytical=Robin_T)
#m.v_z[:] = m.v_z[-1]
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()
ax1.plot(m.T,m.z,c='r')
ax1.set_title('Non-Const. Velocity',fontweight='bold')

m = ice_temperature(Ts=Ts,H=H,adot=adot,eps_xy=eps_xy)
m.flags.pop()
MeyT = Meyer_T(m,T_bulk=0.)
ax2.plot(MeyT,m.z,'-',c='k',lw=2)
m.pmp = np.zeros(m.nz)
m.A_xy_init = 'full'
m.flags.append('plane_strain')
m.flags.append('temp-dependent')
m.initial_conditions(analytical=Robin_T)
m.k = conductivity(m.T.copy(),m.rho)
m.Cp = heat_capacity(m.T.copy())
m.v_z[:] = m.v_z[-1]
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()
ax2.plot(m.T,m.z,c='r')
ax2.set_title('Temp.-Dep. Rate Factor',fontweight='bold')

m = ice_temperature(Ts=Ts,H=H,adot=adot,eps_xy=eps_xy,dS=1.3e-3*180/np.pi)
m.flags.pop()
m.Udef = 40./const.spy
m.Uslide = 140./const.spy
MeyT = Meyer_T(m,T_bulk=0.)
ax3.plot(MeyT,m.z,'-',c='k',lw=2)
m.pmp = np.zeros(m.nz)
m.A_xy_init = 'temperate'
m.flags.append('plane_strain')
m.flags.append('vertical_shear')
m.initial_conditions(analytical=Robin_T)
m.v_z[:] = m.v_z[-1]
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()
ax3.plot(m.T,m.z,c='r')
ax3.set_title('Vertical Shear',fontweight='bold')

m = ice_temperature(Ts=Ts,H=H,qgeo=0.25,adot=adot,eps_xy=eps_xy,dTs=14.65e-6,dH=-2.02e-3,da=-0.48e-6,dS=1.3e-3*180/np.pi)
m.flags.pop()
m.Udef = 40./const.spy
m.Uslide = 140./const.spy
MeyT = Meyer_T(m,T_bulk=0.)
ax4.plot(MeyT,m.z,'-',c='k',lw=2)
m.pmp = np.zeros(m.nz)
m.A_xy_init = 'temperate'
m.flags.append('plane_strain')
m.flags.append('long_advection')
m.initial_conditions(analytical=Robin_T)
m.v_z[:] = m.v_z[-1]
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()
ax4.plot(m.T,m.z,c='r')
ax4.set_title('Longitudinal Advection',fontweight='bold')

m = ice_temperature(Ts=Ts,H=H,qgeo=qgeo,adot=adot,eps_xy=eps_xy,dTs=14.65e-6,dH=-2.02e-3,da=-0.48e-6,dS=1.3e-3*180/np.pi)
m.flags.pop()
m.Udef = 40./const.spy
m.Uslide = 140./const.spy
MeyT = Meyer_T(m,T_bulk=0.)
axn.plot(MeyT,m.z,'-',c='k',lw=2)
m.pmp = np.zeros(m.nz)
m.A_xy_init = 'full'
m.flags.append('plane_strain')
m.flags.append('long_advection')
m.flags.append('vertical_shear')
m.flags.append('temp-dependent')
m.initial_conditions(analytical=Robin_T)
m.k = conductivity(m.T.copy(),m.rho)
m.Cp = heat_capacity(m.T.copy())
#m.v_z[:] = m.v_z[-1]
m.source_terms()
m.stencil(dt=1.*const.spy)
m.numerical_to_steady_state()
axn.plot(m.T,m.z,c='r')
axn.set_title('Total',fontweight='bold')

plt.tight_layout()
plt.savefig('Meyer_comparison.png',dpi=300,facecolor='w')